In [1]:
import sys
sys.path.append('../scripts')  
from models import *
from df_handling import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import warnings
import statistics
warnings.filterwarnings("ignore")

In [2]:
#load training datasets training ta panta plus  postgres
train1 = pd.read_csv('../data/fullfacts_emb.csv')
train2 = pd.read_csv('../data/greek_web_scraping_emb.csv')
train2_1 = pd.read_csv('../data/greek_web_scraping.csv')
train3 = pd.read_csv('../data/snopes2_emb.csv', nrows=1000)
train4 = pd.read_csv('../data/reuters_emb.csv')
train5  = pd.read_csv('../data/test1.csv')
train6 = pd.read_csv('../data/cyprus_factcheck_emb.csv')

train1 = unravel_df(train1, ['text_embedding'])
train2 = unravel_df(train2, ['text_embedding'])
train3 = unravel_df(train3, ['text_embedding'])
train4 = unravel_df(train4, ['text_embedding'])
train5 = unravel_df(train5, ['text_embedding'])
train6 = unravel_df(train6, ['text_embedding'])
train5.rename(columns={'text': 'claim', 'predict_label': 'label'}, inplace=True)


In [3]:
#find whick grek data is from greekhoaxes etc
train2_1 = train2_1[['id','url']]
train2_1['source'] = train2_1['url'].apply(lambda x: 0 if "ellinikahoaxes" in x else (1 if "factcheckgreek" in x else None))
train2 = pd.merge(train2, train2_1[['id','source']], on='id', how='inner')


In [4]:
train1['source'] = 2
train3['source'] = 3
train4['source'] = 4
train5['source'] = 5
train6['source'] = 6



In [5]:
df_train = pd.concat([train1,train2,train3, train4, train5, train6], ignore_index=True)
#keep only 3 labels
df_train = df_train[df_train.label<=2]
df_train.reset_index(drop=True, inplace=True)

In [6]:
NaiveBayes_list  = list()
LR_list  = list()
KNeigh_list  = list()
RF_list  = list()
DT_list  = list()
SVC_list  = list()
MLP_list  = list()
data_source_train_list = list()
data_source_val_list = list()

#list for sub accuracies
NaiveBayes_list_acc  = list()
LR_list_acc  = list()
KNeigh_list_acc  = list()
RF_list_acc  = list()
DT_list_acc  = list()
SVC_list_acc  = list()
MLP_list_acc  = list()

In [7]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for fold, (train_index, valid_index) in enumerate(kf.split(df_train.text_embedding), 1):
    X_train, X_valid = [df_train.text_embedding[i] for i in train_index], [df_train.text_embedding[i] for i in valid_index ]
    y_train, y_valid = df_train.label[train_index] , df_train.label[valid_index]
    data_source_train, data_source_val = df_train.source[train_index], df_train.source[valid_index]
    

    print(f"Fold {fold}:")
    print("Training data:\n ")
    print(data_source_train.value_counts(), '\n')
    print("Validation data:\n ")
    print(data_source_val.value_counts())
    print("---------------------")

    data_source_train_list.append(data_source_train.value_counts())
    data_source_val_list.append(data_source_val.value_counts())

    # nb1, nb2 = cross_val_Naive_Bayes(X_train, y_train, X_valid, y_valid, data_source_val)
    # lr1, lr2 = cross_val_LR(X_train, y_train, X_valid, y_valid, data_source_val)
    # kn1, kn2 = cross_val_Kneigh(X_train, y_train, X_valid, y_valid, data_source_val)
    # svm1, svm2 = cross_val_SVC(X_train, y_train, X_valid, y_valid, data_source_val)
    # rf1, rf2 = cross_val_RF(X_train, y_train, X_valid, y_valid, data_source_val)
    # dt1, dt2 = cross_val_DT(X_train, y_train, X_valid, y_valid, data_source_val)
    mlp1, mlp2 = cross_val_MLP(X_train, y_train, X_valid, y_valid, data_source_val)

    # NaiveBayes_list.append(nb1)
    # NaiveBayes_list_acc.append(nb2)

    # LR_list.append(lr1)
    # LR_list_acc.append(lr2)
 
    # KNeigh_list.append(kn1)
    # KNeigh_list_acc.append(kn2)

    # SVC_list.append(svm1)
    # SVC_list_acc.append(svm2)

    # RF_list.append(rf1)
    # RF_list_acc.append(rf2)

    # DT_list.append(dt1)
    # DT_list_acc.append(dt2)

    MLP_list.append(mlp1)
    MLP_list_acc.append(mlp2)


Fold 1:
Training data:
 
2    1218
3     701
4     342
1     277
0     197
5     135
6     115
Name: source, dtype: int64 

Validation data:
 
2    159
3     58
4     36
1     30
0     22
6     14
5     13
Name: source, dtype: int64
---------------------

Accuracy for data source 2: 0.6981132075471698

Accuracy for data source 0: 0.8636363636363636

Accuracy for data source 1: 0.9666666666666667

Accuracy for data source 3: 0.6551724137931034

Accuracy for data source 4: 0.9444444444444444

Accuracy for data source 5: 0.38461538461538464

Accuracy for data source 6: 1.0
Accuracy for MLP: 0.7530120481927711

[(2, 0.6981132075471698), (0, 0.8636363636363636), (1, 0.9666666666666667), (3, 0.6551724137931034), (4, 0.9444444444444444), (5, 0.38461538461538464), (6, 1.0)]
Fold 2:
Training data:
 
2    1236
3     677
4     342
1     277
0     201
5     135
6     117
Name: source, dtype: int64 

Validation data:
 
2    141
3     82
4     36
1     30
0     18
5     13
6     12
Name: source, dty

In [8]:
# print(statistics.stdev(NaiveBayes_list))
# print("LR: ", statistics.stdev(LR_list))
# print("k-Neigh: ", statistics.stdev(KNeigh_list))
# print("SVC: ", statistics.stdev(SVC_list))
# print("RF: ", statistics.stdev(RF_list))
# print("DT: ", statistics.stdev(DT_list))
print("MLP: ", statistics.stdev(MLP_list))

MLP:  0.02053279205193318


In [9]:
data_source_train_list

[2    1218
 3     701
 4     342
 1     277
 0     197
 5     135
 6     115
 Name: source, dtype: int64,
 2    1236
 3     677
 4     342
 1     277
 0     201
 5     135
 6     117
 Name: source, dtype: int64,
 2    1241
 3     686
 4     342
 1     281
 0     195
 5     130
 6     110
 Name: source, dtype: int64,
 2    1243
 3     683
 4     342
 1     280
 0     185
 5     133
 6     119
 Name: source, dtype: int64,
 2    1246
 3     679
 4     344
 1     266
 0     199
 5     136
 6     115
 Name: source, dtype: int64,
 2    1240
 3     685
 4     337
 1     279
 0     197
 5     133
 6     114
 Name: source, dtype: int64,
 2    1255
 3     665
 4     335
 1     278
 0     202
 5     132
 6     118
 Name: source, dtype: int64,
 2    1229
 3     671
 4     343
 1     278
 0     202
 5     139
 6     124
 Name: source, dtype: int64,
 2    1229
 3     697
 4     337
 1     278
 0     201
 5     131
 6     113
 Name: source, dtype: int64,
 2    1256
 3     687
 4     338
 1     269
 0

In [10]:
df_corr = pd.DataFrame( {'greekhoaxes_train_count':[data_source_train_list[i][0] for i in range (len(data_source_train_list))], 
                        'factcheckgreek_train_count':[data_source_train_list[i][1] for i in range (len(data_source_train_list))],
                        'fullfacts_train_count': [data_source_train_list[i][2] for i in range (len(data_source_train_list))],
                        'snopes_train_count': [data_source_train_list[i][3] for i in range (len(data_source_train_list))],
                        'reuters_train_count': [data_source_train_list[i][4] for i in range (len(data_source_train_list))],
                        'check4facts_train_count': [data_source_train_list[i][5] for i in range (len(data_source_train_list))],
                        'cyprus_train_count': [data_source_train_list[i][6] for i in range (len(data_source_train_list))],
                        'snopes_val_count':[data_source_val_list[i][3] for i in range (len(data_source_val_list))],
                        'greekhoaxes_val_count':[data_source_val_list[i][0] for i in range (len(data_source_val_list))], 
                        'factcheckgreek_val_count':[data_source_val_list[i][1] for i in range (len(data_source_val_list))], 
                        'fullfacts_val_count': [data_source_val_list[i][2] for i in range (len(data_source_val_list))],
                        'reuters_val_count': [data_source_val_list[i][4] for i in range (len(data_source_val_list))],
                        'check4facts_val_count': [data_source_val_list[i][5] for i in range (len(data_source_val_list))],
                        'cyprus_val_count': [data_source_val_list[i][6] for i in range (len(data_source_val_list))],



                        #'NB_grekhoaxes_acc': [t[1][1] for t in NaiveBayes_list_acc], 'NB_factcheck_acc': [t[2][1] for t in NaiveBayes_list_acc],'NB_snopes_acc': [t[3][1] for t in NaiveBayes_list_acc],'NB_fullfacts_acc': [t[0][1] for t in NaiveBayes_list_acc],'NB_reuters_acc': [t[4][1] for t in NaiveBayes_list_acc],'NB_check4facts_acc': [t[5][1] for t in NaiveBayes_list_acc],'NB_cyprus_acc': [t[6][1] for t in NaiveBayes_list_acc],
                        #'LR_grekhoaxes_acc': [t[1][1] for t in LR_list_acc], 'lr_factcheck_acc': [t[2][1] for t in LR_list_acc],'lr_snopes_acc': [t[3][1] for t in LR_list_acc],'lr_fullfacts_acc': [t[0][1] for t in LR_list_acc],'lr_reuters_acc': [t[4][1] for t in LR_list_acc],'lr_check4facts_acc': [t[5][1] for t in LR_list_acc],'lr_cyprus_acc': [t[5][1] for t in LR_list_acc],
                        #'kn_grekhoaxes_acc': [t[1][1] for t in KNeigh_list_acc], 'kn_factcheck_acc': [t[2][1] for t in KNeigh_list_acc],'kn_snopes_acc': [t[3][1] for t in KNeigh_list_acc],'kn_fullfacts_acc': [t[0][1] for t in KNeigh_list_acc],'kn_reuters_acc': [t[4][1] for t in KNeigh_list_acc],'kn_check4facts_acc': [t[5][1] for t in KNeigh_list_acc],'kn_cyprus_acc': [t[5][1] for t in KNeigh_list_acc],
                        #'rf_grekhoaxes_acc': [t[1][1] for t in RF_list_acc], 'rf_factcheck_acc': [t[2][1] for t in RF_list_acc],'rf_snopes_acc': [t[3][1] for t in RF_list_acc],'rf_fullfacts_acc': [t[0][1] for t in RF_list_acc],'rf_reuters_acc': [t[4][1] for t in RF_list_acc],'rf_check4facts_acc': [t[5][1] for t in RF_list_acc],'rf_cyprus_acc': [t[5][1] for t in RF_list_acc],
                        #'dt_grekhoaxes_acc': [t[1][1] for t in DT_list_acc], 'dt_factcheck_acc': [t[2][1] for t in DT_list_acc],'dt_snopes_acc': [t[3][1] for t in DT_list_acc],'dt_fullfacts_acc': [t[0][1] for t in DT_list_acc],'dt_reuters_acc': [t[4][1] for t in DT_list_acc],'dt_check4facts_acc': [t[5][1] for t in DT_list_acc],'dt_cyprus_acc': [t[5][1] for t in DT_list_acc],
                        #'svc_grekhoaxes_acc': [t[1][1] for t in SVC_list_acc], 'svc_factcheck_acc': [t[2][1] for t in SVC_list_acc],'svc_snopes_acc': [t[3][1] for t in SVC_list_acc],'svc_fullfacts_acc': [t[0][1] for t in SVC_list_acc],'svc_reuters_acc': [t[4][1] for t in SVC_list_acc],'svc_check4facts_acc': [t[5][1] for t in SVC_list_acc],'svc_cyprus_acc': [t[5][1] for t in SVC_list_acc],
                        'mlp_greekhoaxes_acc': [t[0][1] for t in MLP_list_acc], 'mlp_factcheckgreek_acc': [t[1][1] for t in MLP_list_acc],'mlp_snopes_acc': [t[2][1] for t in MLP_list_acc],
                        'mlp_fullfacts_acc': [t[0][1] for t in MLP_list_acc],'mlp_reuters_acc': [t[4][1] for t in MLP_list_acc],
                        'mlp_check4facts_acc': [t[3][1] for t in MLP_list_acc],
                        'mlp_cyprus_acc': [t[4][1] for t in MLP_list_acc],











                        #'NB_accuracy': NaiveBayes_list,'LR_accuracy': LR_list,'kn_accuracy': KNeigh_list,'RF_accuracy': RF_list,
                        #'DT_accuracy': DT_list,'SVC_accuracy': SVC_list,
                        'MLP_accuracy': MLP_list})



data = ['train','val']
sources = ['greekhoaxes_','factcheckgreek_','snopes_', 'check4facts_', 'cyprus_','fullfacts_','reuters_']

for s in sources:
    for d in data:
        column = s + d + '_per'
        df_corr[column] = df_corr[s+d+'_count']/(sum(df_corr[s+d+'_count'] for s in sources)) 
        # print(f'''{df_corr[column]}''')


In [11]:
MLP_list_acc[0]

[(2, 0.6981132075471698),
 (0, 0.8636363636363636),
 (1, 0.9666666666666667),
 (3, 0.6551724137931034),
 (4, 0.9444444444444444),
 (5, 0.38461538461538464),
 (6, 1.0)]

In [12]:
from IPython.display import display

for s in sources:
    res = pd.DataFrame(df_corr[['MLP_accuracy', s+'train_per',s+'val_per', 'mlp_'+s+'acc']])
    display(res)

,MLP_accuracy,greekhoaxes_train_per,greekhoaxes_val_per,mlp_greekhoaxes_acc
0,0.753012,0.065997,0.066265,0.698113
1,0.725904,0.067337,0.054217,0.652482
2,0.750000,0.065327,0.072289,0.727941
3,0.746988,0.061977,0.102410,0.738806
4,0.740964,0.066667,0.060241,0.694656
5,0.719880,0.065997,0.066265,0.700730
6,0.743976,0.067672,0.051205,0.737705
7,0.734139,0.067649,0.051360,0.689189
8,0.794562,0.067314,0.054381,0.756757
9,0.734139,0.064300,0.081571,0.685950


,MLP_accuracy,factcheckgreek_train_per,factcheckgreek_val_per,mlp_factcheckgreek_acc
0,0.753012,0.092797,0.090361,0.863636
1,0.725904,0.092797,0.090361,0.944444
2,0.750000,0.094137,0.078313,0.791667
3,0.746988,0.093802,0.081325,0.764706
4,0.740964,0.089112,0.123494,0.650000
5,0.719880,0.093467,0.084337,0.727273
6,0.743976,0.093132,0.087349,0.882353
7,0.734139,0.093101,0.087613,0.882353
8,0.794562,0.093101,0.087613,0.833333
9,0.734139,0.090087,0.114804,0.888889


,MLP_accuracy,snopes_train_per,snopes_val_per,mlp_snopes_acc
0,0.753012,0.234841,0.174699,0.966667
1,0.725904,0.226801,0.246988,0.966667
2,0.750000,0.229816,0.219880,0.923077
3,0.746988,0.228811,0.228916,1.000000
4,0.740964,0.227471,0.240964,0.926829
5,0.719880,0.229481,0.222892,0.964286
6,0.743976,0.222781,0.283133,1.000000
7,0.734139,0.224715,0.265861,0.896552
8,0.794562,0.233423,0.187311,1.000000
9,0.734139,0.230074,0.217523,1.000000


,MLP_accuracy,check4facts_train_per,check4facts_val_per,mlp_check4facts_acc
0,0.753012,0.045226,0.039157,0.655172
1,0.725904,0.045226,0.039157,0.634146
2,0.750000,0.043551,0.054217,0.630137
3,0.746988,0.044556,0.045181,0.592105
4,0.740964,0.045561,0.036145,0.650000
5,0.719880,0.044556,0.045181,0.581081
6,0.743976,0.044221,0.048193,0.617021
7,0.734139,0.046551,0.027190,0.647727
8,0.794562,0.043871,0.051360,0.725806
9,0.734139,0.042867,0.060423,0.555556


,MLP_accuracy,cyprus_train_per,cyprus_val_per,mlp_cyprus_acc
0,0.753012,0.038526,0.042169,0.944444
1,0.725904,0.039196,0.036145,0.916667
2,0.750000,0.036851,0.057229,1.000000
3,0.746988,0.039866,0.030120,1.000000
4,0.740964,0.038526,0.042169,0.941176
5,0.719880,0.038191,0.045181,0.926829
6,0.743976,0.039531,0.033133,0.930233
7,0.734139,0.041527,0.015106,1.000000
8,0.794562,0.037843,0.048338,0.951220
9,0.734139,0.038848,0.039275,0.950000


,MLP_accuracy,fullfacts_train_per,fullfacts_val_per,mlp_fullfacts_acc
0,0.753012,0.408040,0.478916,0.698113
1,0.725904,0.414070,0.424699,0.652482
2,0.750000,0.415745,0.409639,0.727941
3,0.746988,0.416415,0.403614,0.738806
4,0.740964,0.417420,0.394578,0.694656
5,0.719880,0.415410,0.412651,0.700730
6,0.743976,0.420436,0.367470,0.737705
7,0.734139,0.411587,0.447130,0.689189
8,0.794562,0.411587,0.447130,0.756757
9,0.734139,0.420630,0.365559,0.685950


,MLP_accuracy,reuters_train_per,reuters_val_per,mlp_reuters_acc
0,0.753012,0.114573,0.108434,0.944444
1,0.725904,0.114573,0.108434,0.916667
2,0.750000,0.114573,0.108434,1.000000
3,0.746988,0.114573,0.108434,1.000000
4,0.740964,0.115243,0.102410,0.941176
5,0.719880,0.112898,0.123494,0.926829
6,0.743976,0.112228,0.129518,0.930233
7,0.734139,0.114869,0.105740,1.000000
8,0.794562,0.112860,0.123867,0.951220
9,0.734139,0.113195,0.120846,0.950000


In [13]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# correlation_matrix = df_corr.corr()


# for i in range(10,84,10):
#     # Plot the correlation matrix using Seaborn
#     plt.figure(figsize=(20, 15))
#     sns.heatmap(correlation_matrix.iloc[:14,i:i+10], annot=True, cmap='coolwarm', fmt=".4f")
#     plt.title('Correlation Matrix')
#     plt.show()

In [14]:
for t in SVC_list_acc:
    print(t)